# Clustering - Exemplu

Dataset referitor la semințele de grâu: https://archive.ics.uci.edu/ml/datasets/seeds.

Grupul examinat a cuprins semințe aparținând la 3 soiuri diferite de grâu: Kama, Rosa și Canadian. Au existat câte 70 de elemente pentru fiecare, selectate aleator pentru experiment. Vizualizarea de înaltă calitate a structurii interne a miezului seminței a fost detectată utilizând o tehnică X-ray. Aceasta este nedistructivă și considerabil mai ieftină decât alte tehnici pentru imagini, precum microscopia sau tehnologia laser. Imaginile au fost înregistrate pe plăci Kodak X-ray de dimensiune 13 x 18. Au fost realizate studii folosind boabe de grîu recoltate cu combina de pe câmpuri experimentale, apoi au fost explorate la Institutul de agrofizică al Academiei Poloneze de științe din Lublin.

Setul de date poate fi folosit pentru task-uri de clasificare și clustering.

Informații despre atribute:

Pentru a construi setul de date, au fost măsurați 7 parametri geometrici ai semințelor de grâu: 
1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove. 

Toți acești parametri sunt continui, cu valori reale.

Încercăm gruparea acestora în 3 grupuri cu ajutorul metodei K-means.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [ ]:
from pyspark.ml.clustering import KMeans

# Încărcare date.
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [ ]:
dataset.head()

In [ ]:
dataset.describe().show()

## Formatarea datelor

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset.columns

In [ ]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [ ]:
final_data = vec_assembler.transform(dataset)

## Scalarea datelor
Scalarea datelor se poate aplica din motive de dimensionalitate: https://en.wikipedia.org/wiki/Curse_of_dimensionality

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [ ]:
# Calculul statisticilor de sumarizare antrenând StandardScaler
scalerModel = scaler.fit(final_data)

In [ ]:
# Normalizarea fiecărei caracteristici astfel încât să aibă deviația standard unitară.
final_data = scalerModel.transform(final_data)

## Antrenarea și evaluarea modelului

In [ ]:
# Antrenarea unui model k-means.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [ ]:
# Evaluarea clustering-ului 
# Depreciat:
#wssse = model.computeCost(final_data)
#print("Within Set Sum of Squared Errors = " + str(wssse))

#Evaluați modelul




In [ ]:
# Afișare rezultate.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [ ]:
model.transform(final_data).select('prediction').show()